In [1]:
from keras.models import load_model
from keras.layers import *
from keras import Model, Input
from keras import optimizers
import keras.backend as K
import numpy as np
import pickle


def load(filepath):
    with open(filepath, 'rb') as f:
        return pickle.load(f)


dataTriple = load('obj/triplets.pkl')
triplets = dataTriple['triplets'] / 256


/home/henrik/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def get_darknet():
    model = load_model('obj/darknet.h5')
    for layer in model.layers:
        layer.trainable = False
    return model


def make_style_model():
    darknet = get_darknet()

    # pretrained layers
    inputs = darknet.layers[0].input
    x = darknet.layers[16].output

    # new layers
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Lambda(l2_normalize)(x)
    # Embedding model produces style embeddings
    embedding_model = Model(inputs, x)

    # Crate triple model to train the base model
    # inputs for anchor, positive image, and negative image
    input_anc = Input((256, 256, 3))
    input_pos = Input((256, 256, 3))
    input_neg = Input((256, 256, 3))

    # the embedding of the anchor, positive image, and negative image
    embed_anc = embedding_model(input_anc)
    embed_pos = embedding_model(input_pos)
    embed_neg = embedding_model(input_neg)

    # distance between anchor and positive embeddings
    dist_pos = Lambda(sqEucl, output_shape=(1,))([embed_anc, embed_pos])
    # distance between anchor and negative embeddings
    dist_neg = Lambda(sqEucl, output_shape=(1,))([embed_anc, embed_neg])

    # The tiplet_loss: dist_pos - dist_neg + 1 < 0
    triple_loss = Lambda(triplet_loss, output_shape=(1,))([dist_pos, dist_neg])
    # triplet model, for training the embedding model
    triple_model = Model([input_anc, input_pos, input_neg], triple_loss)
    
    style_model = Model([input_anc, input_pos], dist_pos)
    
    return triple_model, style_model


def triplet_loss(x):
    return K.maximum(x[0] - x[1] + 1, 0)


def sqEucl(x):
    return K.sum(K.square(x[0] - x[1]), axis=-1, keepdims=True)

def l2_normalize(x, axis=-1):
    
    square_sum = K.sum(K.square(x), axis, keepdims=True)
    x_inv_norm = 1. / K.sqrt(K.maximum(square_sum, 1e-6))
    return x * x_inv_norm

def identity(y_true, y_pred):
    return K.mean(K.square(y_pred))


In [ ]:
triple_model, style_model = make_style_model()

sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
triple_model.compile(optimizer=sgd, loss=identity)

alphas = np.zeros(triplets.shape[0])
triple_model.fit([triplets[:, 0], triplets[:, 1],
                 triplets[:, 2]], alphas, epochs=20, batch_size=32)



style_model.compile(optimizer=sgd, loss=identity)
style_model.save('obj/style_embed_model.h5', custom_objects={"tf": tf})


/home/henrik/miniconda3/lib/python3.6/site-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Epoch 1/20
1000/1000 [==============================] - 8s 8ms/step - loss: 0.8929
Epoch 2/20
1000/1000 [==============================] - 6s 6ms/step - loss: 0.2372
Epoch 3/20
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0955
Epoch 4/20
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0760
Epoch 5/20
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0723
Epoch 6/20
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0712
Epoch 7/20
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0707
Epoch 8/20
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0705
Epoch 9/20
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0704
Epoch 10/20
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0703
Epoch 11/20
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0702
Epoch 12/20
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0702
E